## **11장 병행성과 네트워크**

<br>

병행성과 네트워크를 활용하는 이유

* `성능`<br>
느린 요소를 기다리지 않고 빠른 요소를 바쁘게 유지

* `견고함`<br>
작업을 복제하여 안정적인 방식으로 운영

* `간소화`<br>
작업을 이해하기 쉽고, 해결하기 쉽도록 여러작업으로 분해

* `커뮤니케이션`<br>
원격으로 데이터를 발신/수신

<br>

## **11.1 병행성**

1. 싱글머신에서 다수의 작업을 빠르게 처리해야할 필요가 발생
2. 각 작업을 독립적으로 만들어 느린작업이 나머지 작업을 블로킹 하지 않도록 구성
3. 모든 작업을 동시에 실행하면 `병목 현상이 발생할 수 있다`
4. 병렬 컴퓨팅의 복잡성을 해결해야할 필요성이 생김

<br>

## **11.1.1 큐**

**`FIFO` (First-In-First-Out)**
![queue](queue_desc.png)

## **11.1.2 프로세스**

`multiprocessing` 모듈을 활용하여 queue 구현

In [ ]:
from multiprocessing import Process, Queue
  
from creator import creator
from consumer import consumer

createQ = Queue()
consumeQ = Queue()

process_create = Process(target=creator, args=(createQ, consumeQ))
process_create.start()

process_consume = Process(target=consumer, args=(consumeQ,))
process_consume.start()

numbers = [1, 2, 3, 4, 5, 'd', -1]
for number in numbers:
    createQ.put(number)


createQ.close()
consumeQ.close()
createQ.join_thread()
consumeQ.join_thread()

process_create.join()
process_consume.join()


<br>

## **11.1.3 쓰레드**

* 쓰레드는 한 프로세스 내에서 실행된다
* 프로세스의 모든 자원에 접근할 수 있다

In [ ]:
import threading, queue
  
from creator_thread import creator
from consumer_thread import consumer

create_queue = queue.Queue()
consume_queue = queue.Queue()

threading.Thread(target=creator, args=(create_queue, consume_queue)).start()
for num in range(2):
    threading.Thread(target=consumer, args=(consume_queue, num)).start()

numbers = [1, 2, 3, 4, 5, -1]
for number in numbers:
    create_queue.put(number)
    



<br>

* threading 모듈에는 terminate() 함수가 없다 - 쉽게 종료할 수 있는 방법이 없음
* 쓰레드는 위험하다 - 매우 찾기 힘든 버그가 발생 할 있다
* 쓰레드를 사용하려면 항상 thread-safe 한 코드를 작성해야 한다<br>
일반적으로 전역데이터를 쓰레드가 변경할때는 lock 을 사용하지만 똑바로 사용하기가 매우 어렵다



<br>

## **11.2 네트워크**

<br>

### **11.2.1 패턴**

* `요청-응답` 패턴<br>
요청을 보낸 후 응답을 기다리는 동기적 방식<br>
ex) 웹브라우저 - 웹서버

* `푸시/팬아웃` 패턴<br>
데이터를 pool 에 있는 사용가능한 워커로 전송<br>
ex) 로드밸런서

* `풀/팬인` 패턴<br>
하나 이상의 소스로부터 데이터를 받는다
ex) 토렌트

* `발행/구독` 패턴<br>
발행자는 데이터 전송만 하고 구독자에 관심이 없음<br>
구독자는 발행자 자체가 아닌 topic 에 관심을 두고 해당 topic의 데이터만 전송받는다<br>
ex) 라디오/텔레비전


<br>

### **11.2.2 발행-구독 모델**

발행-구독은 큐가 아닌 broadcast<br>
* 하나 이상의 프로세스가 메세지를 발행
* 구독자 프로세스는 수신하고자 하는 메세지의 타입을 표시<br>
 --> 각 메세지의 복사본이 타입과 일치하는 구독자에 전송


<br>

### **Redis**

In [ ]:
import threading
from redis_pub import publish
from redis_sub import subscribe

threading.Thread(target=subscribe).start()
threading.Thread(target=publish).start()


<br>

## **11.2.3 TCP/IP**

* `프로토콜`: 커넥션을 맺고, 데이터를 교환하고, 커넥션을 종료 하는 등의 방법에 대한 **규칙**

* 인터넷은 `계층` 으로 정렬되어 있다

* `네트워크의 위치`와 데이트 흐름의 `패킷`을 명시하는 `IP 계층`<br>

* IP 계층 에서 네트워크 사이에서 바이트를 이동하는 프로토콜 두가지<br>
    
    **1. `UDP`(User Datagram Protocol)**<br>
    짧은 데이터 교환에 사용된다. 한 단위 (single burst)로 전송되는 작은 메세지<br>
    `응답 메세지(ACK)가 없기 때문에` 목적지에 잘 도착했는지 확인할 수 없다
    
    **1. `TCP`(Transmission Control Protocol)**<br>
    수명이 긴 커넥션에 사용된다. 바이트 스트림이 중복없이 순서대로 도착하는 것을 보장<br>
    송/수신자 사이의 커넥션을 보장하기 위한 `핸드셰이크`를 설정한다<br>
    웹, 데이터베이스 서버 등 대부분의 인터넷은 TCP 프로토콜로 수행된다



<br>

## **11.2.4 소켓**

#### 네트워크 프로그래밍의 가장 낮은 수준

<br>

### UDP 프로토콜 사용

In [ ]:
import threading
from udp_server import open_server
from udp_client import connect_server
import time

threading.Thread(target=open_server).start()
time.sleep(3)
threading.Thread(target=connect_server).start()


<br>

### TCP 프로토콜 사용

TCP는 클라이언트와 서버의 커넥션을 유지하고 클라이언트의 IP 주소를 기억한다

In [ ]:
import threading
from tcp_server import open_server
from tcp_client import connect_server
import time

threading.Thread(target=open_server).start()
time.sleep(3)
threading.Thread(target=connect_server).start()


<br>

#### TCP 와 UDP 특징들

* UDP 는 메세지 크기에 제한이 있다.
* UDP 는 메세지가 목적지까지 도달하는 것을 보장하지 않는다
* TCP 는 메세지가 아닌 바이트 스트림을 전송한다
* TCP 를 통해 전체 메세지를 전달하기 위해 세그먼트로부터 전체 메세지를 재구성하기 위한 몇가지 추가정보가 필요

![tcp-segmen](desc_tcp_segment.png)


<br>

### **DNS**

DNS(Domain Name System)는 분산된 데이터베이스를 통해<br>
IP 주소 -> 이름 또는 그반대를 수행하는 인터넷 서비스

In [ ]:
import socket

# 호스트이름 -> ip 주소
socket.gethostbyname('www.google.com')

In [ ]:
# 호스트이름 -> 이름, 또 다른 호스트이름 리스트, ip주소 리스트
socket.gethostbyname_ex('www.google.com')


<br>

## **11.2.8 웹 서비스와 API**


#### 웹사이트에서 데이터에 대한 API를 제공할 때 사용방법

In [ ]:
import requests

url = "https://jsonplaceholder.typicode.com/todos/1"

# 데이터 얻어옴
response = requests.get(url)

# json 으로 parsing
response.json()


<br>

## **11.2.9 원격 프로세싱**

`RPC`(Remote Procedure Call)는 네트워크를 통해 원격에 있는 머신을 실행시킨다

<br>

#### 원격 클라이언트 측
1. 함수의 인자를 바이트로 변환
2. 인코딩된 바이트를 원격 머신으로 전송

#### 원격 머신 측
1. 인코딩된 요청 바이트를 수신
2. 바이트 수신후 다시 원래의 데이터 구조로 바이트를 디코딩
3. 디코딩된 데이터와 함께 로컬 함수를 찾아서 호출
4. 함수 결과를 인코딩
5. 클라이언트에게 인코딩된 바이트 전송

In [1]:
from rpc_serer import open_rpc_server
from rpc_client import connect_rpc
import threading
import time

threading.Thread(target=open_rpc_server).start()
time.sleep(3)
threading.Thread(target=connect_rpc).start()

server started


127.0.0.1 - - [26/Jul/2020 11:10:02] "POST / HTTP/1.1" 200 -


log complete: please


127.0.0.1 - - [26/Jul/2020 11:10:05] "POST / HTTP/1.1" 200 -


log complete: log


127.0.0.1 - - [26/Jul/2020 11:10:08] "POST / HTTP/1.1" 200 -


log complete: this


127.0.0.1 - - [26/Jul/2020 11:10:11] "POST / HTTP/1.1" 200 -


log complete: message
